# Inputs and Targets

In [4]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [7]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [12]:
data_preprocessed['Excessive Absenteeism'] = targets


In [19]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'], axis=1)


In [20]:
unscaled_inputs = data_with_targets.iloc[:,:14]

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy=copy,with_mean=with_mean,with_std=with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.array(np.mean(X[self.columns]))
        self.var_ = np.array(np.var(X[self.columns]))
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [34]:
unscaled_inputs.columns.values

array(['Reasons_diseases', 'Reasons_pregnancy',
       'Reasons_health_symptomps', 'Reasons_light', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [40]:
unscaled_columns = unscaled_inputs.iloc[:, :4]
columns_to_scale = ['Month Value', 'Transportation Expense', 'Age', 'Body Mass Index', 'Education', 'Children', 'Pets']

In [41]:
scaler = CustomScaler(columns=columns_to_scale)
scaler.fit(unscaled_inputs)
scaled_inputs = scaler.transform(unscaled_inputs)

/Users/khaleed_mammad/Desktop/Khalid/Data Science/Projects/Absenteeism - Complete/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:4006: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [42]:
# scaled_inputs = pd.DataFrame(scaled_inputs, columns=columns_to_scale.columns)
scaled_inputs.head()

,Reasons_diseases,Reasons_pregnancy,Reasons_health_symptomps,Reasons_light,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1,1.005844,36,-0.536062,239.554,0.767431,-0.44798,0.880469,0.268487
1,0,0,0,0,0.182726,1,-1.574681,13,2.130803,239.554,1.002633,-0.44798,-0.019280,-0.589690
2,0,0,0,1,0.182726,2,-0.654143,51,0.248310,239.554,1.002633,-0.44798,-0.919030,-0.589690
3,1,0,0,0,0.182726,3,0.854936,5,0.405184,239.554,-0.643782,-0.44798,0.880469,-0.589690
4,0,0,0,1,0.182726,3,1.005844,36,-0.536062,239.554,0.767431,-0.44798,0.880469,0.268487


In [43]:
scaled_inputs = pd.concat([unscaled_columns, scaled_inputs], axis=1)

In [44]:
scaled_inputs.head()

,Reasons_diseases,Reasons_pregnancy,Reasons_health_symptomps,Reasons_light,Reasons_diseases,Reasons_pregnancy,Reasons_health_symptomps,Reasons_light,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0,0,0,1,0.182726,1,1.005844,36,-0.536062,239.554,0.767431,-0.44798,0.880469,0.268487
1,0,0,0,0,0,0,0,0,0.182726,1,-1.574681,13,2.130803,239.554,1.002633,-0.44798,-0.019280,-0.589690
2,0,0,0,1,0,0,0,1,0.182726,2,-0.654143,51,0.248310,239.554,1.002633,-0.44798,-0.919030,-0.589690
3,1,0,0,0,1,0,0,0,0.182726,3,0.854936,5,0.405184,239.554,-0.643782,-0.44798,0.880469,-0.589690
4,0,0,0,1,0,0,0,1,0.182726,3,1.005844,36,-0.536062,239.554,0.767431,-0.44798,0.880469,0.268487


# Splitting into Train and Test

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, shuffle = True, random_state = 20)

# Logistic Regression

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 

In [48]:
reg = LogisticRegression()
reg.fit(X_train, y_train)

/Users/khaleed_mammad/Desktop/Khalid/Data Science/Projects/Absenteeism - Complete/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [49]:
reg.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [52]:
reg.score(X_train, y_train)

0.7589285714285714

In [51]:
reg.intercept_

array([-0.00402586])

In [53]:
reg.coef_

array([[ 1.13778174, -0.02695073,  1.52867576,  0.12989583,  1.13778174,
        -0.02695073,  1.52867576,  0.12989583,  0.137903  , -0.02496546,
         0.70847397, -0.00722609, -0.37168609, -0.00318949,  0.45347348,
        -0.18932854,  0.46705158, -0.36974651]])

In [54]:
feature_names = scaled_inputs.columns.values
feature_names

array(['Reasons_diseases', 'Reasons_pregnancy',
       'Reasons_health_symptomps', 'Reasons_light', 'Reasons_diseases',
       'Reasons_pregnancy', 'Reasons_health_symptomps', 'Reasons_light',
       'Month Value', 'Day of the Week', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets'], dtype=object)

In [57]:
summary_table = pd.DataFrame(columns=['Feature_names'], data = feature_names)
summary_table

,Feature_names
0,Reasons_diseases
1,Reasons_pregnancy
2,Reasons_health_symptomps
3,Reasons_light
4,Reasons_diseases
5,Reasons_pregnancy
6,Reasons_health_symptomps
7,Reasons_light
8,Month Value
9,Day of the Week


In [59]:
summary_table['Cofficients'] = np.transpose(reg.coef_)
summary_table

,Feature_names,Cofficients
0,Reasons_diseases,1.137782
1,Reasons_pregnancy,-0.026951
2,Reasons_health_symptomps,1.528676
3,Reasons_light,0.129896
4,Reasons_diseases,1.137782
5,Reasons_pregnancy,-0.026951
6,Reasons_health_symptomps,1.528676
7,Reasons_light,0.129896
8,Month Value,0.137903
9,Day of the Week,-0.024965


In [60]:
summary_table.index = summary_table.index + 1
summary_table

,Feature_names,Cofficients
1,Reasons_diseases,1.137782
2,Reasons_pregnancy,-0.026951
3,Reasons_health_symptomps,1.528676
4,Reasons_light,0.129896
5,Reasons_diseases,1.137782
6,Reasons_pregnancy,-0.026951
7,Reasons_health_symptomps,1.528676
8,Reasons_light,0.129896
9,Month Value,0.137903
10,Day of the Week,-0.024965


In [62]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

In [41]:
summary_table.sort_index()

,Feature_names,Cofficients
0,Intercept,-1.739645
1,Reasons_diseases,2.860885
2,Reasons_pregnancy,0.682125
3,Reasons_health_symptomps,3.198483
4,Reasons_light,0.891279
5,Month Value,0.164352
6,Transportation Expense,0.615001
7,Age,-0.175882
8,Body Mass Index,0.281843
9,Education,-0.096914


In [63]:
summary_table['Odds_ratio'] = np.exp(summary_table['Cofficients'])

In [64]:
summary_table.sort_values('Odds_ratio', ascending = False)

,Feature_names,Cofficients,Odds_ratio
3,Reasons_health_symptomps,1.528676,4.612065
7,Reasons_health_symptomps,1.528676,4.612065
1,Reasons_diseases,1.137782,3.119840
5,Reasons_diseases,1.137782,3.119840
11,Transportation Expense,0.708474,2.030890
17,Children,0.467052,1.595284
15,Body Mass Index,0.453473,1.573769
9,Month Value,0.137903,1.147864
4,Reasons_light,0.129896,1.138710
8,Reasons_light,0.129896,1.138710


In [65]:
# If the weight is around 0 (or Odd Ratio is around 1), these inputs are not important. We can delete them
# In our case -> Daily Work Load Average, Distance to Work, Day of the Week

# Testing the Model

In [66]:
reg.score(X_test, y_test)

0.7071428571428572

In [67]:
predicted_probability = reg.predict_proba(X_test) 

In [68]:
predicted_probability

array([[0.75284227, 0.24715773],
       [0.64249687, 0.35750313],
       [0.34647245, 0.65352755],
       [0.78263075, 0.21736925],
       [0.05984988, 0.94015012],
       [0.3026796 , 0.6973204 ],
       [0.27187595, 0.72812405],
       [0.07178535, 0.92821465],
       [0.78849195, 0.21150805],
       [0.75565525, 0.24434475],
       [0.46258146, 0.53741854],
       [0.10820328, 0.89179672],
       [0.04561425, 0.95438575],
       [0.71580448, 0.28419552],
       [0.31922666, 0.68077334],
       [0.4983955 , 0.5016045 ],
       [0.49939344, 0.50060656],
       [0.61206759, 0.38793241],
       [0.50052029, 0.49947971],
       [0.01950932, 0.98049068],
       [0.70888954, 0.29111046],
       [0.77835368, 0.22164632],
       [0.41708396, 0.58291604],
       [0.42927134, 0.57072866],
       [0.14202886, 0.85797114],
       [0.79231453, 0.20768547],
       [0.42246426, 0.57753574],
       [0.90980246, 0.09019754],
       [0.14696236, 0.85303764],
       [0.77401674, 0.22598326],
       [0.

In [69]:
# First columns shows probability our model assigned to 0; vice versa for column 2

# Saving My Model

In [71]:
# import pickle
# with open('scaler', 'wb') as file:
#     pickle.dump(scaler, file)
# with open('scaler', 'wb') as file:
#     pickle.dump(scaler, file)
